## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [1]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy


model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size = (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(200, activation='relu')) 
model.add(Dense(10, activation='softmax'))#nombre de classes

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "sgd")

X_train = X_train.reshape(60000, 28, 28, 1) / 255 #MinMaxScaling en divisant par le max parce que le min est égal à 0

model.fit(X_train,y_train)

1875/1875 [==============================] - 21s 4ms/step - loss: 0.3015


Afficher le nombre de paramètre du modèle

In [2]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (32, 26, 26, 32)          320       
                                                                 
 conv2d_1 (Conv2D)           (32, 24, 24, 64)          18496     
                                                                 
 flatten (Flatten)           (32, 36864)               0         
                                                                 
 dense (Dense)               (32, 200)                 7373000   
                                                                 
 dense_1 (Dense)             (32, 10)                  2010      
                                                                 
Total params: 7,393,826
Trainable params: 7,393,826
Non-trainable params: 0
_________________________________________________________________
None


Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [3]:
import joblib
import os

model_file_name = "mnist_.joblib"
joblib.dump(model, model_file_name)

print("taille du fichier :", os.path.getsize('/content/mnist_.joblib'), "octets")
print("taille par paramètre : 59177771 / 7393826 = ", 59177771  / 7393826, "octets")

taille du fichier : 59177443 octets
taille par paramètre : 59177771 / 7393826 =  8.003673740766958 octets


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [4]:
converter = tensorflow.lite.TFLiteConverter.from_keras_model(model)

Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [5]:
model.tflite = converter.convert()

model_tflitefile_name = "mnistLite_.joblib"
joblib.dump(model.tflite, model_tflitefile_name)

print("taille du fichier lite :", os.path.getsize('/content/mnistLite_.joblib'), "octets")
print("Le fichier est 2 fois moins gros")

taille du fichier lite : 29578297 octets
Le fichier est 2 fois moins gros


On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [6]:
optimized_converter = tensorflow.lite.TFLiteConverter.from_keras_model(model)
optimized_converter.optimizations = [tensorflow.lite.Optimize.DEFAULT]

optimized_model = optimized_converter.convert()
model_optimized_name = "model_optimize.joblib"
joblib.dump(optimized_model, model_optimized_name)

print("taille du fichier lite :", os.path.getsize('/content/model_optimize.joblib'), "octets")
print("Le fichier est très largement plus léger")

taille du fichier lite : 7399513 octets
Le fichier est très largement plus léger


Quelle type  de quantization Optimize.Default, utilise-t-elle ?


## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. 

Reprendre le modèle entraîné sur MNIST


In [7]:
loaded_model = joblib.load('/content/mnist_.joblib')

A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

Compiler le modèle

Afficher le summury du modèle. D'après vous ce modèle est-il quantifié ? 

Réentraîner votre modèle sur un sous ensemble des modèles sur une ou deux epochs et afficher la performance sur le train et test set

Convertir votre modèle avec TFLite

In [8]:
!pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 63.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [9]:
qat_model = tfmot.quantization.keras.quantize_model(loaded_model)

qat_model.compile(loss = "sparse_categorical_crossentropy", optimizer = "sgd")

print(qat_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (32, 28, 28, 1)          3         
 er)                                                             
                                                                 
 quant_conv2d (QuantizeWrapp  (32, 26, 26, 32)         387       
 erV2)                                                           
                                                                 
 quant_conv2d_1 (QuantizeWra  (32, 24, 24, 64)         18627     
 pperV2)                                                         
                                                                 
 quant_flatten (QuantizeWrap  (32, 36864)              1         
 perV2)                                                          
                                                                 
 quant_dense (QuantizeWrappe  (32, 200)                7

In [10]:
qat_model.fit(X_train,y_train,epochs=2,validation_data=(X_test, y_test))

Epoch 1/2
1875/1875 [==============================] - 26s 13ms/step - loss: 0.1367 - val_loss: 0.1379
Epoch 2/2
1875/1875 [==============================] - 18s 9ms/step - loss: 0.1007 - val_loss: 0.1380


In [11]:
print("Evaluate on test data")
results = qat_model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

Evaluate on test data
313/313 [==============================] - 1s 4ms/step - loss: 0.1380
test loss, test acc: 0.13801495730876923


In [12]:
qat_converter = tensorflow.lite.TFLiteConverter.from_keras_model(qat_model)
tflite_qat_model = qat_converter.convert()

Comparer la performance du modèle Quantified aware training, au modèle original et au modèle quantifié post training

Sauvegarder le modèle QAT et comparer les tailles des modèles

In [13]:
joblib.dump(tflite_qat_model, "QAT_model.joblib")

print("taille du fichier :", os.path.getsize('/content/QAT_model.joblib'), "octets")

taille du fichier : 29584201 octets


Bonus : déployer votre modèle sur votre téléphone ou un dispositif embarqué si vous en disposez d'un. 

Bonus : Obtenir un modèle qui sera à la fois quantifié et élagué (prunned) en s'aidant de la documentation (https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)

A l'aide de tensorflow lite / tensorflow lite micro 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d51e245-899d-41d6-b23b-cf3e4bbbc6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>